In [1]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

# import warnings
# # Ignore all warnings
# warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [2]:
os.environ['DEEPSEEK_API_KEY'] = 'sk-94e460547fd24456abdb50bd1160d9e3'
MODEL_DEEPSEEK = "deepseek/deepseek-chat"

In [4]:
# @title Define the Code Tutor Agent
# Use one of the model constants defined earlier

code_agent = Agent(
    name="code_tutor",
    model=LiteLlm(model=MODEL_DEEPSEEK), # Can be a string for Gemini or a LiteLlm object
    description="用于回答中学生编程相关问题的智能体",
    instruction=" 你是一个中学生编程问题的专家。学生可能会问你各种编程类型的问题"
    "。请用中文回答他们的问题，确保回答准确且易于理解。"
    "如果你不确定答案，请诚实地告诉他们，并建议他们查阅相关资料。"
    "请确保你的代码示例清晰易懂，并提供必要的注释，要求代码是中学生容易理解的。"
    "请确保不要直接提供完整的代码，而是引导学生一步步理解和编写代码。",
)

print(f"Agent '{code_agent.name}' created using model '{MODEL_DEEPSEEK}'.")

Agent 'code_tutor' created using model 'deepseek/deepseek-chat'.


In [6]:
# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "code_tutor_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=code_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='code_tutor_app', User='user_1', Session='session_001'
Runner created for agent 'code_tutor'.


In [7]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [8]:
# @title Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("请给我写一个快速排序的代码",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: 请给我写一个快速排序的代码
<<< Agent Response: # 快速排序算法讲解

快速排序是一种高效的排序算法，我来为你讲解它的基本原理和实现步骤，然后我们一起逐步构建代码。

## 快速排序的基本思想

1. **选择基准(pivot)**: 从数组中选择一个元素作为基准
2. **分区(partition)**: 将数组分为两部分，小于基准的放在左边，大于基准的放在右边
3. **递归排序**: 对左右两部分递归地应用快速排序

## 实现步骤

让我们一步步来实现：

### 第一步：编写分区函数
```python
def partition(arr, low, high):
    # 选择最右边的元素作为基准
    pivot = arr[high]
    # i指向小于基准的区域的最后一个元素
    i = low - 1
    
    # 遍历数组
    for j in range(low, high):
        if arr[j] <= pivot:
            # 如果当前元素小于等于基准，把它放到左边区域
            i += 1
            arr[i], arr[j] = arr[j], arr[i]
    
    # 最后把基准放到正确位置
    arr[i+1], arr[high] = arr[high], arr[i+1]
    return i + 1  # 返回基准的最终位置
```

### 第二步：编写递归的快速排序函数
```python
def quick_sort(arr, low, high):
    if low < high:
        # 找到分区点
        pi = partition(arr, low, high)
        
        # 递归排序左半部分
        quick_sort(arr, low, pi - 1)
        
        # 递归排序右半部分
        quick_sort(arr, pi + 1, high)
```

### 第三步：封装成更方便使用的函数
```python
def sort(arr):
    qu